Supposons que nous soyons à la tête de wikipédia et que nous décidions d'en faire un projet lucratif en utilisant le comportement des utilisateurs·rice·s à des fins publicitaires. Pour que les utilisateurs·rice·s passent plus de temps sur wikipédia, nous allons mettre en place un système d'amis qui se basera sur les centres d'intérêts communs. En d'autre terme, nous allons créer un réseau social centré sur la connaissance et les centres d'intérêts.

Wikipédia possède déjà toutes les données dont on pourrait rêver pour notre projet. Ici, nous allons utiliser un fichier nommé ``smallwikipedia.csv``, qui contient un petit échantillon de l'historique des modifications d'articles en rapport avec la révolution française. Nous allons rassembler des données pour recommander des ami·e·s au sein de notre wikipédia lucratif.

Partie 1
-----------------------

Les fichiers ``.csv`` sont essentiellement des tableaux, dont les colonnes sont représentées par les points-virgules ``;`` entre les données. Ils peuvent être lus dans un tableur.

Ouvrez le fichier ``smallwikipedia.csv`` dans Excel ou Open Office Calc et comparez le résultat avec l'affichage des ``6`` première lignes du fichier en exécutant la cellule ci-après.

**Question 1** 
À quoi correspondent les deux premières lignes du fichier (voir la partie sur les données du cours) ?


**Question 1 Réponse** : La première ligne correspond aux descripteurs la seconde ligne aux types des valeurs des données stockées dans notre fichier.

In [85]:
f=open("smallwikipedia.csv","r")
print(f.readline())
print(f.readline())
print(f.readline())
print(f.readline())
print(f.readline())
print(f.readline())

User;Name;Date;changes

Category;Category;Date[dd/MM/yyyy HH:mm];int

194.210.1.51;Bonnet_phrygien;19/10/2004 11:02;163

Ske;Bonnet_phrygien;19/10/2004 12:11;751

Ske;Bonnet_phrygien;19/10/2004 12:12;17

83.27.202.42;Bonnet_phrygien;21/10/2004 10:39;42



Nous observons qu'un même article peut être modifié par plusieurs personnes. La taille de la modification (et donc le temps passé)  correspond à la quatrième colonne. Nous pourrions donc recommander aux utilisateurs d'ajouter en amis d'autres utilisateurs qui se sont intéressés et ont modifié les même articles qu'eux.

Ici, les données récupérées sont sous forme de grandes chaînes de caractères. Cette organisation ne nous permet pas de récupérer facilement nos données. Il serait plus simple d'avoir à disposition une liste par colonne du fichier. Ici, les dates de modification ne nous intéressent pas, nous souhaitons donc récupérer 3 colonnes : ``utilisateurs``,``articles`` et ``modifs``.

Pour cela, nous allons tout d'abord stocker chacune des lignes du fichier dans une longue liste. Nous allons utiliser une boucler ``for``. Celle-ci va effectuer la même opération pour chaque ligne ``ligne`` de notre fichier. L'opération ``append`` d'une liste permet d'ajouter un nouvel élément à la fin de la liste.

Ainsi, pour chaque nouvelle ligne du fichier, nous allons l'ajouter à la fin de la ligne : 

In [86]:
fichier = open("smallwikipedia.csv","r")
lignes = [] ### lignes est pour le moment une liste vide

for ligne in fichier :  # pour chaque ligne du fichier, on l'ajoute à lignes
    lignes.append(ligne)
    
print(len(lignes)) #on affiche le nombre d'éléments de la liste lignes

1000


**Question 2** 
Les deux premières lignes ne correspondent pas à des données, il nous faut donc les supprimer de notre tableau. Pour cela, utilisez la fonction ``ma_liste.pop(i)`` qui supprime le i ème élément de la liste (attention on compte à partir de ``0``).

**Question 3**
Afficher la 141 ème ligne du fichier.

In [87]:
lignes.pop(0)
lignes.pop(0)
print(lignes[140])

Vargenau;Coup_d'Ã‰tat_du_18_fructidor_an_V;29/01/2006 12:48;135



Pour chaque ligne du fichier, nous allons maintenant séparer la chaîne de caractère en 4, grâce aux poits-virgules. Nous allons pour cela utiliser à nouveau la boucle ``for``, mais cette fois avec un indice ``i`` qui va prendre toutes les valeurs possible entre ``0`` et la taille de la liste.

In [88]:
for i in range(0,len(lignes)) :
    lignes[i] = lignes[i].split(';')

**Question 4** Afficher la 141ème ligne à nouveau. Que s'est-il passé ? Expliquer.

In [89]:
print(lignes[140])

['Vargenau', "Coup_d'Ã‰tat_du_18_fructidor_an_V", '29/01/2006 12:48', '135\n']


**Question 4 Réponse :** Chaque ligne, au lieu d'être une chaîne de caractère (donc du type *string*) est maintenant une liste de chaînes de caractères. Nous avons coupé note chaîne de caractère au niveau des points-virgules ';'. Cela va nous permettre d'accéder à chaque colonne de notre petite base de données.

Nous allons maintenant faire en sorte que nos données soient facilement accessibles. Cela commence par convertir le type dela taille des modifications des chaînes de caractère vers les entiers.

In [90]:
for i in range(0,len(lignes)) :
    lignes[i][3][:-1] ## On supprime les retours à la ligne '\n'
    lignes[i][3] = int(lignes[i][3])

**Question 5** Même question que la **Question 4**

**Question 5 Réponse :** La dernière colonne de notre tableau est toujours une chaîne de caractère (string), alors que nous savons qu'elle est sensée être du type entier (int). On commence donc par retirer le caractère de retour à la ligne, puis on convertit vers le type entier. Cela va nous permettres d'avoir accès à toutes les opérations disponibles sur les entiers (addition, multiplication, comparaison, etc.)

In [91]:
print(lignes[142])

['Vicomte Moniot', 'Cocarde_tricolore', '02/02/2006 05:08', 519]


**Question 6** 
Supprimez les dates de modifications sur toutes les lignes

In [92]:
for i in range(0,len(lignes)) :
    lignes[i].pop(2)

In [93]:
print(lignes[140])

['Vargenau', "Coup_d'Ã‰tat_du_18_fructidor_an_V", 135]


Partie 2
----------------------
Nous allons à présent mettre en place un algorithme rudimentaires pour calculer un score de compatibilité entre deux utilisateur·rice·s.

Commençons par calculer la quantité totale de modifications d'un·e utilisateur·rice.

In [94]:
def nb_modif(utilisateur) : # Le fonctionnement de cette fonction doit être expliqué avec des commentaires (qui commencent par #)
    modifs = 0 #on initialise notre compteur de modifications à 0
    for ligne in lignes : # On parcourt toutes les lignes de notre base de données
        if ligne[0] == utilisateur : # Si la ligne parcourue concerne l'utilisateur qui nous intéresse
            modifs = modifs + ligne[2] # Alors on ajoute les modification associées à notre compteur de modifications
    return modifs #on renvoie le nombre total de modification de l'utilisateur
    
print(nb_modif("Ske"))

768


**Question 7** Expliquez avec des commentaires le fonctionnement du code ci-dessus.

La compatibilité d'un·e utilisateur·rice ``u1`` avec un·e autre ``u2`` peut être considéré comme la quantité de modifications qu'``u1`` a effectuées sur un article où ``u2`` a aussi effectué une modification, divisée par le nombre total de modification d'``u1``.

En d'autre terme, si on note ``m1`` la quantité totale de modifications d'``u1``.

Nous allons commencer par créer la liste de tous les utilisateur·rice·s.



In [95]:
utilisateurs = [] # on initialise une liste vide d'utilisateurs
for ligne in lignes : # On parcourt chaque ligne de notre base de données
    if not (ligne[0] in utilisateurs) : # on vérifie que l'utilisateur de la ligne courante n'est pas déjà dans notre liste
        utilisateurs.append(ligne[0]) # On ajoute notre utilisateur à notre liste
    

**Question 8** Expliquez le code ci-dessus avec des commentaires

**Question 9** Combien y a-t-il d'utilisateurs différents ? 

In [96]:
print("Il y a " + str(len(utilisateurs)) + " utilisateurs différents.")

Il y a 321 utilisateurs différents.


**Question 10** On considère le graphe dont les nœuds sont les utilisateur·rice·s, et dont les arêtes correspondent aux modifications communes entre deux utilisateurs. Si nous voulons stocker ce graphe sous la forme d'une matrice d'adjacence, combien de booléens devrions nous stocker ? Est-ce optimal ? Proposer une alternative.

**Question 10 Réponse** Avec une matrice d'adjacence, il nou faudrait stocker $321*321$ entiers (int) différents. Cependant, la plupart de ces entiers vaudront 0. On pourrait donc se contenter de créer la liste des listes des contributions communes de chaque utilisateur.  Une telle liste est appelée une liste d'adjacence.

In [97]:
utilisateurs.sort()

La liste des utilisateur·rice·s est maintenant ordonnée. 

**Question 11** Afficher l'avant-dernier·ère utilisateur·rice. Qu'observe-t-on ?  Quel était probablement sont réel pseudo ? Comment expliquer un tel problème d'affichage ?



Avec Jupyter Notebook, un problème d'encodage apparaît. Voir la fiche d'exercices sur les chaînes de caractères.

In [98]:
print(utilisateurs[-2])

Ã‰clusette


**Question 12** Créez la liste du nombre total de modification par utilisateur· que vous appellerez ``total_modifs``. Vous pourrez vous appuyer sur le fonction ``nb_modif``.

In [99]:
total_modifs = []
for utilisateur in utilisateurs :
    total_modifs.append(nb_modif(utilisateur))

Pour chaque utilisateur, nous allons commencer par calculer la liste des article auquel il a contribué, ainsi que le nombre de modifications associé à chaque article.  Nous représenterons donc le couple d'un article ``a`` et de son nombre de modifications ``m`` associées comme des liste de deux éléments ``[a,m]``.



In [100]:
def contributions(utilisateur) : 
    contribs = [] #on initialise notre liste avec la liste vide
    for ligne in lignes : # on parcourt chaque ligne de notre base de données
        if ligne[0] == utilisateur : # on ne s'occcupe que de l'utilisateur qui nous intéresse
            deja_contribue = False # on va vérifié si on a déjà l'article de la ligne dans contribs
            for contrib in contribs : # on parcourt toutes les contributions qu'on a trouvées jusqu'à présent
                if contrib[0] == ligne[1] : #si on avait déjà trouvé une contributions sur le même article
                    deja_contribue = True # on actualise notre booléen
                    contrib[1] += ligne[2] # on ajoute la contribution à notre paire déjà existante
                    break
            if not deja_contribue : # si on n'a pas trouvé l'article dans contribs
                contribs.append([ligne[1],ligne[2]]) # on crée une nouvelle paire qu'on ajoute à contribs
    return contribs        
    

**Question 13** Expliquer le code ci-dessus à l'aide de commentaires.

**Question 14** Afficher les contributions de l'utilisateur·rice ``Richelieu``.

In [101]:
contributions("Richelieu")

[["ComitÃ©_d'instruction_publique", 1349],
 ['Adoption_par_Louis_XVI_de_la_cocarde_tricolore', 1478],
 ['Bataille_de_Cholet', 2839],
 ['Complot_rÃ©volutionnaire_nantais_contre_les_modÃ©rÃ©s', 122],
 ['Alexandre_FranÃ§ois_Marie_de_Beauharnais', 953],
 ["Coup_d'Ã‰tat_du_30_prairial_an_VII_(18_juin_1799)", 1768],
 ["Coup_d'Ã‰tat_du_18_fructidor_an_V", 2016],
 ["Complot_de_l'Å’illet", 4704],
 ['Conspiration_de_BicÃªtre', 4858],
 ['Conseil_exÃ©cutif_(RÃ©volution_franÃ§aise)', 976],
 ['Club_des_Impartiaux', 567],
 ['Club_de_1789', 717],
 ['Club_de_Salm', 559],
 ['Bon_Albert_Briois_de_Beaumetz', 3623],
 ['Certificat_de_civisme', 579]]

Notre fonction de recommendation d'amis est maintenant à portée de main : pour chaque utilisateur·rice, nous avons la liste des articles auxquel iel a contribué. 

**Question 16** 
En utilisant la fonction ``contributions`` et la fonction ``nb_modifs``, implémentez la fonction ``afinite`` qui prend en argument deux utilisateurs et qui renvoie l'affinité du premier utilisateurs avec le second utilisateur.

In [102]:
def affinite(utilisateur1, utilisateur2) :
    contribs1 = contributions(utilisateur1) #on récupère les contributions des deux utilisateurs
    contribs2 = contributions(utilisateur2)
    modifs_communes = 0
    for contrib1 in contribs1 : 
        for contrib2 in contribs2 : # on récupère tous les couples possible de contribution
            if contrib1[0] == contrib2[0] : # si le nom de l'article est identique, alors c'est un article commun
                modifs_communes += contrib1[1] # on ajoute les contributions de l'utilisateur1 à ses contributions communes
    return modifs_communes / nb_modif(utilisateur1) # on divise les modifications communes par le nombre total de modifications de l'utilisateur
        

**Question 17** Créez la liste des affinités de l'utilisateur·rice ``Richelieu``. 

In [112]:
richelieu = []
for utilisateur in utilisateurs :
    richelieu.append((affinite("Richelieu",utilisateur),utilisateur))
    


**Question 18** Quel est l'utilisateur avec le plus d'affinité avec ``Richelieu`` (hormis Richelieu lui·elle-même) ? 

In [114]:
richelieu.sort(reverse=1)
print(richelieu[:5])

[(1.0, 'Richelieu'), (0.9347794009148591, 'Chicobot'), (0.6972480448576066, 'Vargenau'), (0.41032167625793126, 'Polmars'), (0.4101003393832079, 'PierreEG')]
